# Visualization for Publication

Answering the following research questions:

* How is the increase of the density of the public fast-charging infrastructure network affecting the adoption of battery-eletric vehicles in different income classes?
* Is there a spill-over effect of local public charging infrastructure investments on the adoption of battery-electric passenger 

In [14]:
import matplotlib.pyplot as plt  
import numpy as np
import os
import sys
import pandas as pd
import geopandas as gpd
import yaml

In [15]:
def process_key(key):
    # Convert string key to a tuple
    return safe_tuple_parser(key)

def process_value(value):
    # Convert string value to float
    return float(value)

def safe_tuple_parser(key):
    """
    Safely parses string representations of nested tuples into actual Python tuples.
    Example: "(2024, (1, 7, 0), (1, 8), 2024)" -> (2024, (1, 7, 0), (1, 8), 2024)
    """
    import ast  # Abstract Syntax Tree module for safe literal evaluation

    # Remove outer quotes if present and use `ast.literal_eval`
    try:
        return ast.literal_eval(key)
    except (SyntaxError, ValueError) as e:
        raise ValueError(f"Failed to parse key: {key}") from e


In [16]:
def read_data(case_study_name, input_file_name):
    current_path = os.getcwd()
    print("Current path:", current_path)
    file_results = os.path.normpath(current_path + "/results")
    print("File results:", os.path.normpath(file_results))
    file_path = os.path.join(current_path, "/results")
    print(file_path)
    # Normalize the path
    normalized_path = os.path.normpath(file_path)
    print("Normalized path:", normalized_path)

    # reading input data 
    folder_input = os.path.normpath(current_path + "/Basque country/data")
    with open(folder_input + "/" + input_file_name) as file:
        input_data = yaml.safe_load(file)

    with open(os.path.normpath(file_results + "/" + case_study_name + "_n_fueling_dict.yaml")) as file:
        n_fueling_dict= yaml.safe_load(file)

    with open(os.path.normpath(file_results + "/" + case_study_name + "_budget_penalty_minus_dict.yaml")) as file:
        budget_penalty_minus_dict = yaml.safe_load(file)

    with open(os.path.normpath(file_results + "/" + case_study_name + "_budget_penalty_plus_dict.yaml")) as file:
        budget_penalty_plus_dict= yaml.safe_load(file)

    with open(os.path.normpath(file_results + "/" + case_study_name + "_detour_time_dict.yaml")) as file:
        detour_time_dict= yaml.safe_load(file)

    with open(os.path.normpath(file_results + "/" + case_study_name + "_f_dict.yaml")) as file:
        f_dict= yaml.safe_load(file)

    with open(os.path.normpath(file_results + "/" + case_study_name + "_h_dict.yaml")) as file:
        h_dict= yaml.safe_load(file)

    with open(os.path.normpath(file_results + "/" + case_study_name + "_h_exist_dict.yaml")) as file:
        h_exist_dict= yaml.safe_load(file)

    with open(os.path.normpath(file_results + "/" + case_study_name + "_h_minus_dict.yaml")) as file:
        h_minus_dict= yaml.safe_load(file)

    with open(os.path.normpath(file_results + "/" + case_study_name + "_h_plus_dict.yaml")) as file:
        h_plus_dict= yaml.safe_load(file)

    with open(os.path.normpath(file_results + "/" + case_study_name + "_q_fuel_infr_plus_dict.yaml")) as file:
        q_fuel_infr_plus_dict= yaml.safe_load(file)

    with open(os.path.normpath(file_results + "/" + case_study_name + "_q_mode_infr_plus_dict.yaml")) as file:
        q_mode_infr_plus_dict= yaml.safe_load(file)

    with open(os.path.normpath(file_results + "/" + case_study_name + "_s.yaml")) as file:
        s_dict= yaml.safe_load(file)
    
    budget_penalty_minus = {process_key(key): process_value(value) for key, value in budget_penalty_minus_dict.items()}
    budget_penalty_plus = {process_key(key): process_value(value) for key, value in budget_penalty_plus_dict.items()}
    detour_time = {process_key(key): process_value(value) for key, value in detour_time_dict.items()}
    f = {process_key(key): process_value(value) for key, value in f_dict.items()}
    h = {process_key(key): process_value(value) for key, value in h_dict.items()}
    h_exist = {process_key(key): process_value(value) for key, value in h_exist_dict.items()}
    h_minus = {process_key(key): process_value(value) for key, value in h_minus_dict.items()}
    h_plus = {process_key(key): process_value(value) for key, value in h_plus_dict.items()}
    q_fuel_infr_plus = {process_key(key): process_value(value) for key, value in q_fuel_infr_plus_dict.items()}
    q_mode_infr_plus = {process_key(key): process_value(value) for key, value in q_mode_infr_plus_dict.items()}
    s = {process_key(key): process_value(value) for key, value in s_dict.items()}
    n_fueling = {process_key(key): process_value(value) for key, value in n_fueling_dict.items()}

    output_data = {"budget_penalty_minus": budget_penalty_minus, "budget_penalty_plus": budget_penalty_plus, "detour_time": detour_time, "f": f, "h": h, "h_exist": h_exist, "h_minus": h_minus, "h_plus": h_plus, "q_fuel_infr_plus": q_fuel_infr_plus, "q_mode_infr_plus": q_mode_infr_plus, "s": s, "n_fueling": n_fueling}

    return input_data, output_data
    


## Plotting detour time 

In [17]:
case_studies = ["testing_FAST_2025-02-11_14-45-35"]
input_file_name = "transport_data_years_v65_FAST_DENSIFICATION.yaml"
case_study_output = {}
case_study_input = {}
for cs in case_studies:
    input_data, output_data = read_data(cs, input_file_name)
    case_study_output[cs] = output_data
    case_study_input[cs] = input_data


Current path: c:\Github\iDesignRES_transcompmodel\examples
File results: c:\Github\iDesignRES_transcompmodel\examples\results
c:/results
Normalized path: c:\results


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Github\\iDesignRES_transcompmodel\\examples\\results\\testing_FAST_2025-02-11_14-45-35_n_fueling_dict.yaml'

In [ ]:
detour_times_at_edge = {}
model_parameters = input_data["Model"]
Y = model_parameters["Y"]
y_init = model_parameters["y_init"]
pre_y = model_parameters["pre_y"]
product_list = input_data["Product"]
odpair_list = input_data["Odpair"]
path_list = input_data["Path"]
techvehicle_list = input_data["TechVehicle"]
fuel_list = input_data["Fuel"]
geometric_element_list = input_data["GeographicElement"]

odpair_list_dict = {item["id"]: item for item in odpair_list}
techvehicle_list_dict = {item["id"]: item for item in techvehicle_list}
product_list_dict = {item["id"]: item for item in product_list}
path_list_dict = {item["id"]: item for item in path_list}
fuel_list_dict = {item["id"]: item for item in fuel_list}
geometric_element_list_dict = {item["id"]: item for item in geometric_element_list}


G = pre_y + Y
g_init = y_init - pre_y
Y_end = y_init + Y - 1

geom_id = 12
fuel_id = 2
for cs in case_studies:
    detour_time = case_study_output[cs]["detour_time"]
    n_fueling = case_study_output[cs]["n_fueling"]
    for y in range(y, Y_end + 1):
        detour_times_sum = sum([detour_time[(y, (p, r, odpair_list[r]["path_id"], geom_id), fuel_id)] for r in odpair_list_dict.keys() for k in odpair_list_dict for p in product_list_dict.keys() if (y, (p, r, odpair_list[r]["path_id"], geom_id), fuel_id) in detour_time.keys()])    
        n_fueling_sum = sum([n_fueling[(y, (p, r, odpair_list[r]["path_id"], geom_id), fuel_id)] for r in odpair_list_dict.keys() for k in odpair_list_dict for p in product_list_dict.keys() if (y, (p, r, odpair_list[r]["path_id"], geom_id), fuel_id) in n_fueling.keys()])    
        detour_times_at_edge[(y, fuel_id)] = detour_times_sum / n_fueling_sum

print(detour_times_at_edge)
         


NameError: name 'input_data' is not defined